<a href="https://colab.research.google.com/github/BrunoRBorges/Challenge_Data_Science/blob/main/Data_science_s01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Challenge Data Science Alura - Semana 01

Bem-vindo ao meu GitHub! Meu nome é Bruno, e gostaria de lhe apresentar um pouco do trabalho que desenvolvi durante este Challenge!
  


#Problema

A imobiliária InsightPlaces, situada na cidade do Rio de Janeiro, está enfrentando dificuldades para alugar e vender imóveis. Em uma pesquisa de como empresas semelhantes operam no mercado, a InsightPlaces percebeu que esse problema pode estar relacionado aos valores dos imóveis e às recomendações que faz.

Dentro desse contexto, como podemos definir de forma eficiente os preços dos imóveis lidando com grande volume de dados? É importante recomendar imóveis utilizando outro critério? O que precisa ser feito?

Faço parte do time de Ciência de Dados e Big Data da InsightPlaces e fiquei responsável por auxiliar no processo de análise de dados dos imóveis localizados em alguns bairros da cidade do Rio de Janeiro.

Esse projeto tem algumas etapas como: ler e fazer o tratamento do histórico dos preços de imóveis no Rio de Janeiro, construir um modelo de regressão para precificar imóveis e criar um recomendador de imóveis. Para cada uma dessas etapas vou utilizar a ferramenta PySpark que oferece uma melhor performance ao trabalharmos com grandes volumes de dados.

In [1]:
# Instala o PySpark na máquina virtual
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 38 kB/s 
     |████████████████████████████████| 199 kB 59.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ec067750577bfe42c917327fb68465cc31589f77ce11c1f2adb1624ef21f9128
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
# Importa a sessão Spark e os SparkTypes
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, DoubleType
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Cria a sessão Spark (lembrar de fechar a sessão depois)
spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Challenge Semana 01") \
    .getOrCreate()

#Arquivo de Dados  .json

A InsightPlaces disponibilizou os dados contidos no banco de dados quanto às informações de preços de imóveis da cidade do Rio de Janeiro. Subi este arquivo em meu drive conforme abaixo:

In [4]:
# Cria um dataset Spark com as informações brutas do arquivo json
dataset_imoveis_bruto = spark.read.json('/content/drive/MyDrive/Challenge Data Science/dataset_bruto.json')

In [31]:
# Conta a quantidade de dados no json

dataset_imoveis_bruto.printSchema()
dataset_imoveis_bruto.count()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

89083

# Quebrando os arrays do .json

O arquivo .json deixou tudo agrupado em grandes arrays, como visto no printSchema acima, para conseguir trabalhar com isso, transformando num dataframe legal, primeiramente vou quebrar as informações do anúncio, ignorando as colunas de imagem e usuário.

In [6]:
# Cria colunas para cada um dos dados contidos em anuncio
dataset_anuncio = dataset_imoveis_bruto.select('anuncio.*')
dataset_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

# Tratando as colunas de anúncio

Estamos próximo de um dataframe trabalhável, mas os dados ainda estão todos dentro de arrays também, irei trabalhar com as colunas para retirar esses valores de dentro de colunas.

In [8]:
# Transforma as colunas com unidades diferenciadas em suas unidades corretas
dataset_anuncio = dataset_anuncio\
  .withColumn('area_total',dataset_anuncio['area_total'][0].cast(IntegerType()))\
  .withColumn('area_util',dataset_anuncio['area_util'][0].cast(IntegerType()))\
  .withColumn('banheiros',dataset_anuncio['banheiros'][0])\
  .withColumn('quartos',dataset_anuncio['quartos'][0])\
  .withColumn('suites',dataset_anuncio['suites'][0])\
  .withColumn('vaga',dataset_anuncio['vaga'][0])

In [10]:
# Realiza contagem por tipo anuncio
dataset_anuncio\
  .select('tipo_anuncio')\
  .groupBy('tipo_anuncio')\
  .count()\
  .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|88827|
|  Lançamento|  256|
+------------+-----+



# Análise dos tipos de Anúncio

Neste dataframe, possuímos 88827 imóveis usados registrados, contra 256 lançamentos, devido a baixa quantidade de lançamentos, irei retirá-los de nosso estudo.

In [11]:
# Filtra somente por imoveis usados
dataset_anuncio = dataset_anuncio\
  .select('*')\
  .where('tipo_anuncio=="Usado"')

In [12]:
# Realiza contagem por tipo anuncio após filtro
dataset_anuncio\
  .select('tipo_anuncio')\
  .groupBy('tipo_anuncio')\
  .count()\
  .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|88827|
+------------+-----+



In [13]:
# Realiza contagem por tipo anuncio
dataset_anuncio\
  .select('tipo_uso')\
  .groupBy('tipo_uso')\
  .count()\
  .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4539|
|Residencial|84288|
+-----------+-----+



# Análise de imóveis por tipo de uso

Possuímos 4539 imóveis comerciais contra 84288 residenciais, novamente, pela baixa quantidade de dados, irei focar a análise nos imóveis residenciais.

In [14]:
# Filtra somente por imoveis residenciais
dataset_anuncio = dataset_anuncio\
  .select('*')\
  .where('tipo_uso=="Residencial"')

In [15]:
# Realiza contagem por tipo anuncio após filtro
dataset_anuncio\
  .select('tipo_uso')\
  .groupBy('tipo_uso')\
  .count()\
  .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|Residencial|84288|
+-----------+-----+



In [16]:
# Realiza contagem por tipo unidade
dataset_anuncio\
  .select('tipo_unidade')\
  .groupBy('tipo_unidade')\
  .count()\
  .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros| 7502|
| Apartamento|66562|
|        Casa|10224|
+------------+-----+



# Análise por tipo de unidade

Possuímos 10224 casas, 7502 "outros" e 66562 apartamentos, este relatório irá se focar somente nos imóvei do tipo apartamento, excluindo casas e outros.

In [17]:
# Filtra somente por apartamentos
dataset_anuncio = dataset_anuncio\
  .select('*')\
  .where('tipo_unidade=="Apartamento"')

In [18]:
# Realiza contagem por tipo unidade após filtro
dataset_anuncio\
  .select('tipo_unidade')\
  .groupBy('tipo_unidade')\
  .count()\
  .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
| Apartamento|66562|
+------------+-----+



# Separando as informações contidas dentro do array de endereço e de valores

O endereço ainda continha um array contendo Bairro e Zona, e valores ainda era um array com valor, tipo, condomínio e iptu, vou discriminá-los em suas próprias coluna, mas da coluna valor irei extrair somente o valor do imóvel.

In [22]:
# Retira as informações de endereço necessárias, assim como as de valor
dataset_anuncio_filtrado = dataset_anuncio\
  .select('andar','area_total','area_util','banheiros','caracteristicas','endereco.bairro','endereco.zona','id','quartos','suites','tipo_anuncio','tipo_unidade','tipo_uso','vaga','valores.valor')

In [24]:
# Converte a informação de valor em inteiro
dataset_anuncio_filtrado = dataset_anuncio_filtrado\
  .withColumn('valor',dataset_anuncio_filtrado['valor'][0].cast(IntegerType()))

# Salvando a base de dados

Aqui crio um arquivo .parque para registar todos esses filtros realizados.

In [25]:
# Cria o output num arquivo Parquet
dataset_anuncio_filtrado.write.parquet(
    path = '/content/drive/MyDrive/Challenge Data Science/Semana 1/output_parquet',
    mode = 'overwrite',
)

# Teste de leitura

Após salvar o arquivo, subo ele novamente no colab para um teste de leitura.

In [28]:
# Lendo o arquivo Parquet para ter certeza que foi tudo certo

dataset_parquet = spark.read.parquet('/content/drive/MyDrive/Challenge Data Science/Semana 1/output_parquet/part-00000-0781f6f2-bb6a-4e85-bfd2-7825fd885cf3-c000.snappy.parquet')
dataset_parquet.show()

+-----+----------+---------+---------+--------------------+--------------------+----------+--------------------+-------+------+------------+------------+-----------+----+-----+
|andar|area_total|area_util|banheiros|     caracteristicas|              bairro|      zona|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|valor|
+-----+----------+---------+---------+--------------------+--------------------+----------+--------------------+-------+------+------------+------------+-----------+----+-----+
|    3|        43|       43|        1|[Academia, Churra...|           Paciência|Zona Oeste|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|15000|
|    2|        42|       42|        1|[Churrasqueira, P...|           Paciência|Zona Oeste|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|15000|
|    1|        41|       41|        1|[Portaria 24h, Co...|           Guaratiba|Zona Oeste|18d22cbe-1b86-476...|   

In [29]:
# Encerra a sessão Spark
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7f5a3ed8f760>>